<a href="https://colab.research.google.com/github/CauchyLee1981/MediapipeHandtracking_GPU_Bitmap_Input/blob/master/%E2%80%9CwordToText_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [0]:
import os
import numpy as np
from keras.utils import np_utils


def my_generator(path="data", batch_size=50, n_classes=2, frame_num=32, shift=8):
    key_point=25
    channel=3
    
    if not os.path.exists(path):
        print(path, "is not exist\n")
        return

    train_data = []
    counter = 0
    for root, folders, _ in os.walk(path): break
    print(root,folders)
    while True:
      for folder in folders:
        for root2, _, files in os.walk(os.path.join(root, folder)): break
        print("\n",root2,files)
        for file in files:
          with open(os.path.join(root2, file)) as tmp_file:
            a = tmp_file.read()
            b = np.array(a.split()).reshape(-1, key_point, channel)
            c = b[:,:,:2]
            b = c
            for i in range(5):
              start = i * shift +frame_num
              end = start + frame_num
              X = b[start:end]
              X = X.reshape(1, frame_num, key_point, channel-1)
              X1 = b[(start+1):(end+1)]
              X1 = X1.reshape(1, frame_num, key_point, channel-1)
              y = np_utils.to_categorical(int(folder), num_classes=n_classes)
              y = y.reshape(1, len(y))
              yield [X,X1], y



gen = my_generator()
i = 0

for X,y in gen:
  print(X)
  #print(y)
  if i >= 50: 
    break
  else:
    i=i+1


In [0]:
import time

import argparse
import json
import os

from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import *
from keras.layers.convolutional import *
from keras.layers.core import *
from keras.models import Model
from keras.optimizers import adam


def one_obj(frame_l=32, joint_n=25, joint_d=3, words_size=500):
    input_joints = Input(name='joints', shape=(frame_l, joint_n, joint_d))
    input_joints_diff = Input(name='joints_diff', shape=(frame_l, joint_n, joint_d))

    x = Conv2D(filters=64, kernel_size=(1, 1), padding='same')(input_joints)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # @LeakyReLU()(x)

    x = Conv2D(filters=32, kernel_size=(3, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)

    x = Permute((1, 3, 2))(x)

    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(x)
    x = LeakyReLU()(x)  # BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)

    x_d = Conv2D(filters=64, kernel_size=(1, 1), padding='same')(input_joints_diff)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)  # LeakyReLU()(x_d)

    x_d = Conv2D(filters=32, kernel_size=(3, 1), padding='same')(x_d)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)  # LeakyReLU()(x_d)

    x_d = Permute((1, 3, 2))(x_d)

    x_d = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x_d)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)  # LeakyReLU()(x_d)
    x_d = MaxPool2D(pool_size=(2, 2))(x_d)
    x_d = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(x_d)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)  # LeakyReLU()(x_d)
    x_d = MaxPool2D(pool_size=(2, 2))(x_d)

    x = concatenate([x, x_d], axis=-1)

    x = Conv2D(filters=128, kernel_size=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.1)(x)

    x = Conv2D(filters=256, kernel_size=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.1)(x)
    x = Flatten()(x)
    x = Dense(256)(x)
    x = LeakyReLU()(x)  # LeakyReLU()(x)
    x = Dense(words_size, activation='softmax')(x)
    return Model([input_joints, input_joints_diff], x)


model = one_obj(32, 25, 3, 2)
opt = adam(0.005, epsilon=1e-8)
model.compile(opt, 'categorical_crossentropy', metrics=['acc'])
model.summary()

early = EarlyStopping(monitor="loss", mode="min", patience=10)
lr_change = ReduceLROnPlateau(monitor="loss", verbose=1, factor=0.2, patience=100, min_lr=1e-8, cooldown=100)
lr_change.set_model(model)
lr_change.on_train_begin()
best_acc = 0.0







Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
joints_diff (InputLayer)        (None, 32, 25, 3)    0                                            
__________________________________________________________________________________________________
joints (InputLayer)             (None, 32, 25, 3)    0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 32, 25, 64)   256         joints_diff[0][0]                
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 32, 25, 64)   256         joints[0][0]                     
____________________________________________________________________________________________

In [0]:

train_generator = my_generator()

# Train model on dataset
model.fit_generator(generator=train_generator,
          steps_per_epoch=50,
          epochs=20)

Epoch 1/20
data ['000', '001']

 data/000 ['P01_01_00_4._body.txt', 'P01_01_00_0._body.txt', 'P01_01_00_3._body.txt', 'P01_01_00_2._body.txt', 'P01_01_00_1._body.txt']
13/50 [======>.......................] - ETA: 6s - loss: 0.0469 - acc: 1.0000
 data/001 ['P01_01_01_4._body.txt', 'P01_01_01_0._body.txt', 'P01_01_01_2._body.txt', 'P01_01_01_3._body.txt', 'P01_01_01_1._body.txt']
39/50 [======================>.......] - ETA: 0s - loss: 5.8016 - acc: 0.6410
 data/000 ['P01_01_00_4._body.txt', 'P01_01_00_0._body.txt', 'P01_01_00_3._body.txt', 'P01_01_00_2._body.txt', 'P01_01_00_1._body.txt']
50/50 [==============================] - 3s 68ms/step - loss: 8.0712 - acc: 0.5000
Epoch 2/20
13/50 [======>.......................] - ETA: 0s - loss: 1.1921e-07 - acc: 1.0000
 data/001 ['P01_01_01_4._body.txt', 'P01_01_01_0._body.txt', 'P01_01_01_2._body.txt', 'P01_01_01_3._body.txt', 'P01_01_01_1._body.txt']
39/50 [======================>.......] - ETA: 0s - loss: 5.7860 - acc: 0.6410
 data/000 ['P0

In [0]:
import numpy as np
 
a = np.array([[1,2,3],
        [3,4,5],
        [4,5,6]])
print(a)
# 从某个索引处开始切割
print('从数组索引 a[1:] 处开始切割')
print(a[1:,1:])

[[1 2 3]
 [3 4 5]
 [4 5 6]]
从数组索引 a[1:] 处开始切割
[[4 5]
 [5 6]]
